In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the DeepSolar Data and Fields List

In [2]:
solar=pd.read_csv('../../deepsolar_tract.csv',encoding = "ISO-8859-1")
solar.head()
solar_fields=pd.read_csv('../deepsolar fields.csv')
solar_fields.head()

,Field,Description,Unit,Data Type,Formula,Possible Values,Observed Max,Observed Min,Theoretical Min,Theoretical Max,Relevant Feature,Mentioned in Supplemental Info
0,Unnamed: 0,Index,NaN,Numeric,NaN,NaN,72537.000,0,NaN,NaN,0,NaN
1,tile_count,total number of tiles in census tract,NaN,Numeric,NaN,NaN,4468.000,0,0.0,NaN,0,NaN
2,solar_system_count,Total number of solar systems in census tract,NaN,Numeric,NaN,NaN,1535.000,0,0.0,NaN,0,NaN
3,total_panel_area,NaN,NaN,Numeric,NaN,NaN,592031.075,0,0.0,NaN,0,NaN
4,fips,FIPS identifier for the census tract,NaN,String,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


# Load/Test PySpark

In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
import numpy as np

TOTAL = 100
dots = sc.parallelize([2.0 * np.random.random(2) - 1.0 for i in range(TOTAL)]).cache()
print("Number of random points:", dots.count())

stats = dots.stats()
print('Mean:', stats.mean())
print('stdev:', stats.stdev())

Number of random points: 100
Mean: [-0.04298605 -0.05369376]
stdev: [ 0.56574788  0.61382167]


# Train Model Using Spark ML

In [5]:
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.sql.session import SparkSession
from pyspark.sql import *
from pyspark.sql.types import *
spark = SparkSession(sc)
from pyspark.ml.linalg import DenseVector
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

## Pre-process for Spark ML

In [7]:
features=solar_fields.loc[(solar_fields['Mentioned in Supplemental Info']==1)]['Field'].tolist()
#all_variables=['number_of_solar_system_per_household']features+

#Set infinity and blank spaces to NaN in independent variables, set infinite to N/A in dependent variable
solar2=solar[features].replace([np.inf,' '],np.nan)
solar2['number_of_solar_system_per_household']=solar['number_of_solar_system_per_household'].replace(np.inf,np.nan)

#remove census tracts where number of solar systems per household is N/A
solar2=solar2.loc[np.isfinite(solar2['number_of_solar_system_per_household'])]
#Create binary version of number_of_solar_system_per_household for RF classifier

solar2['solar_flag']=solar2['number_of_solar_system_per_household'].apply(lambda x: int(x>0))


## Write solar2 to csv - cleaner than trying to convert pandas df to pyspark df

In [16]:
solar2.to_csv('ml_frame.csv',index=False)

## Read csv into pyspark df

In [17]:
df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("ml_frame.csv")

In [18]:
df.printSchema()

root
 |-- average_household_income: double (nullable = true)
 |-- gini_index: double (nullable = true)
 |-- heating_fuel_coal_coke: integer (nullable = true)
 |-- heating_fuel_electricity: integer (nullable = true)
 |-- heating_fuel_fuel_oil_kerosene: integer (nullable = true)
 |-- heating_fuel_gas: integer (nullable = true)
 |-- heating_fuel_housing_unit_count: integer (nullable = true)
 |-- heating_fuel_none: integer (nullable = true)
 |-- heating_fuel_other: integer (nullable = true)
 |-- heating_fuel_solar: integer (nullable = true)
 |-- population_density: double (nullable = true)
 |-- education_less_than_high_school_rate: double (nullable = true)
 |-- education_high_school_graduate_rate: double (nullable = true)
 |-- education_college_rate: double (nullable = true)
 |-- education_bachelor_rate: double (nullable = true)
 |-- education_master_rate: double (nullable = true)
 |-- education_professional_school_rate: double (nullable = true)
 |-- education_doctoral_rate: double (nullab

## Process Spark Df for Random Forest Classifier - Create Train and Test Sets

In [19]:
cols=df.columns
indep_vars=[i for i in cols if i not in ['solar_flag','number_of_solar_system_per_household']]
classifier_cols=['solar_flag']+indep_vars
regressor_cols=['number_of_solar_system_per_household']+indep_vars

In [20]:
(trainingData, testData) = df.randomSplit([0.8, 0.2],seed=1234)

classifier_train=trainingData.select(classifier_cols).rdd.map(lambda x: (x[0], DenseVector(x[1:])))
classifier_test=testData.select(classifier_cols).rdd.map(lambda x: (x[0], DenseVector(x[1:])))

regressor_train=trainingData.select(regressor_cols).rdd.map(lambda x: (x[0], DenseVector(x[1:])))
regressor_test=testData.select(regressor_cols).rdd.map(lambda x: (x[0], DenseVector(x[1:])))

In [21]:
classifier_train_df = spark.createDataFrame(classifier_train, ["label", "features"])
classifier_test_df = spark.createDataFrame(classifier_test, ["label", "features"])


regressor_train_df=spark.createDataFrame(regressor_train, ["label", "features"])
regressor_test_df=spark.createDataFrame(regressor_test, ["label", "features"])

### Generate Full classifier and regressor frames as reference for featureIndexer

In [22]:
classifier_data=df.select(classifier_cols).rdd.map(lambda x: (x[0], DenseVector(x[1:])))
regressor_data=df.select(regressor_cols).rdd.map(lambda x: (x[0], DenseVector(x[1:])))

classifier_df = spark.createDataFrame(classifier_data, ["label", "features"])
regressor_df=spark.createDataFrame(regressor_data, ["label", "features"])

## Create Model Pipeline - Classifier

In [23]:
#index the labels and the features for the random forest model
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(classifier_df)
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(classifier_df)


#define the model and the label converter
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=100)
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

#put all the steps together into a pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

#fit the training data
model = pipeline.fit(classifier_train_df)

#generate predictions on the test data
predictions = model.transform(classifier_test_df)

## Evaluate Classifier Accuracy

In [24]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print('test set accuracy: ',accuracy)

test set accuracy:  0.7894993724724585


## Get Classifier Feature Importances

In [25]:
rfModel = model.stages[2]
feature_importances=rfModel.featureImportances
#print(len(indep_vars))
fia=feature_importances.toArray()
#print(fia)
fi_list=[]
for i in range(fia.shape[0]):
    fi_list.append((indep_vars[i],fia[i]))
sorted_list=sorted(fi_list,reverse=True,key=lambda k: k[1])
sorted_list[0:20]

[('population_density', 0.15785987050546338),
 ('race_asian_rate', 0.07311933877696758),
 ('property_tax', 0.058303934142207206),
 ('occupancy_vacant_rate', 0.0478366617429207),
 ('housing_unit_median_gross_rent', 0.042898528563938332),
 ('relative_humidity', 0.03941090221677257),
 ('number_of_years_of_education', 0.036149394913741799),
 ('housing_unit_median_value', 0.035461007572820974),
 ('education_high_school_graduate_rate', 0.034961937478504686),
 ('heating_fuel_coal_coke', 0.032808727714627436),
 ('heating_fuel_housing_unit_count', 0.031608582059085628),
 ('education_bachelor_rate', 0.029686302116606701),
 ('daily_solar_radiation', 0.028876156184724722),
 ('occupation_agriculture_rate', 0.021468553851796142),
 ('mortgage_with_rate', 0.020707050742682444),
 ('rebate', 0.018561267257728449),
 ('transportation_public_rate', 0.018163798987350918),
 ('sales_tax', 0.015188612517060718),
 ('heating_fuel_gas', 0.014448301216708039),
 ('education_master_rate', 0.013950959255079456)]

## Create Model Pipeline - RandomForestRegressor

In [26]:
#define the feature indexer
featureIndexer2 =VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(regressor_df)


#define the model
rf2 = RandomForestRegressor(featuresCol="indexedFeatures", numTrees=200)

#put all the steps together into a pipeline
pipeline2 = Pipeline(stages=[featureIndexer2, rf2])

#fit the training data
model2 = pipeline2.fit(regressor_train_df)

#generate predictions on the test data
predictions2 = model2.transform(regressor_test_df)

In [27]:
predictions2.show(50)

+--------------------+--------------------+--------------------+--------------------+
|               label|            features|     indexedFeatures|          prediction|
+--------------------+--------------------+--------------------+--------------------+
|       1.33333333333|[NaN,NaN,0.0,0.0,...|[NaN,NaN,0.0,0.0,...|  0.8656985836412305|
|                 1.0|[NaN,NaN,0.0,0.0,...|[NaN,NaN,0.0,0.0,...| 0.10170528964910887|
| 0.33333333333299997|[NaN,NaN,0.0,0.0,...|[NaN,NaN,0.0,0.0,...| 0.13230157653725272|
|       1.23076923077|[NaN,NaN,0.0,0.0,...|[NaN,NaN,0.0,0.0,...|  0.6329449031357716|
|      0.166666666667|[NaN,0.0572,0.0,1...|[NaN,0.0572,0.0,1...|0.058312821851019055|
|                 0.0|[NaN,0.0890000000...|[NaN,0.0890000000...| 0.09163164591759874|
|                 0.0|[NaN,0.168,0.0,0....|[NaN,0.168,0.0,0....| 0.07450030941297642|
|      0.111111111111|[NaN,0.2061,0.0,0...|[NaN,0.2061,0.0,0...| 0.08441218001502536|
| 0.14285714285700002|[9040.0,0.1867,0....|[9040.0,0.1

## Assess R^2 for Regression Model

In [28]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions2)
print("R^2 on test data = %g" % r2)

R^2 on test data = 0.30543


## Regression Model Feature Importances

In [29]:
rfModel2 = model2.stages[1]
feature_importances2=rfModel2.featureImportances
#print(len(indep_vars))
fia2=feature_importances2.toArray()
#print(fia)
fi_list2=[]
for i in range(fia2.shape[0]):
    fi_list2.append((indep_vars[i],fia2[i]))
sorted_list2=sorted(fi_list2,reverse=True,key=lambda k: k[1])
sorted_list2[0:20]

[('property_tax', 0.10033256465810428),
 ('heating_fuel_gas', 0.055154644739344602),
 ('occupancy_owner_rate', 0.052282131837348922),
 ('net_metering', 0.050491575044885668),
 ('voting_2016_gop_percentage', 0.041976757424097733),
 ('relative_humidity', 0.040971851362740494),
 ('occupation_manufacturing_rate', 0.039064399644131304),
 ('voting_2016_dem_percentage', 0.037987050315577177),
 ('average_household_income', 0.034741546886339833),
 ('rebate', 0.033927390864932121),
 ('feedin_tariff', 0.031437251717809339),
 ('housing_unit_median_gross_rent', 0.025176275499740369),
 ('population_density', 0.02382833386768245),
 ('age_more_than_85_rate', 0.018243868561410723),
 ('heating_fuel_electricity', 0.015726719117490785),
 ('air_temperature', 0.014858009041496171),
 ('cooling_degree_days', 0.014851941123538175),
 ('household_type_family_rate', 0.014701945276339994),
 ('earth_temperature', 0.013613236428957122),
 ('occupation_agriculture_rate', 0.013389656965433171)]

## Overall R^2 for SolarForest

In [30]:
classifier_preds=predictions.select('prediction').collect()
classifier_preds_list=[i.prediction for i in classifier_preds]

In [31]:
regressor_preds=predictions2.select('prediction').collect()
regressor_preds_list=[i.prediction for i in regressor_preds]

In [32]:
def new_preds(classifier,regressor):
    new_preds=[]
    for i in range(len(classifier)):
        if classifier[i]==0:
            new_preds.append(0)
        else:
            new_preds.append(regressor[i])
            
    return new_preds

In [33]:
new_predictions_list=new_preds(classifier_preds_list,regressor_preds_list)
actual_vals=predictions2.select('label').collect()
actual_vals_list=[i.label for i in actual_vals]

In [34]:
from sklearn.metrics import r2_score

In [35]:
r2_score(actual_vals_list,new_predictions_list)

0.008011237456744924

# Generate List of Actual and Predicted Values for Entire Dataset

## Run the Model on the Full Dataset.  Just Use RF Regression for Now

In [36]:
model_final = pipeline2.fit(regressor_df)

## Examine Feature Importances for the whole Dataset

In [38]:
rfModel_final = model_final.stages[1]
feature_importances_final=rfModel_final.featureImportances
#print(len(indep_vars))
fia_final=feature_importances_final.toArray()
#print(fia)
fi_list_final=[]
for i in range(fia_final.shape[0]):
    fi_list_final.append((indep_vars[i],fia_final[i]))
sorted_list_final=sorted(fi_list_final,reverse=True,key=lambda k: k[1])
sorted_list_final[0:20]

[('property_tax', 0.096026236173719762),
 ('occupation_manufacturing_rate', 0.069349349958601744),
 ('occupancy_owner_rate', 0.058916898930618411),
 ('net_metering', 0.054056804903429873),
 ('voting_2016_dem_percentage', 0.043146977032435002),
 ('heating_fuel_gas', 0.042131925774413978),
 ('relative_humidity', 0.037114361370008342),
 ('average_household_income', 0.035201182045460577),
 ('feedin_tariff', 0.03348104711922309),
 ('rebate', 0.032585525844226179),
 ('population_density', 0.028959726914829065),
 ('housing_unit_median_gross_rent', 0.021754373575866459),
 ('voting_2016_gop_percentage', 0.021620701697655675),
 ('daily_solar_radiation', 0.018230781365257052),
 ('cooling_degree_days', 0.014558921556898656),
 ('heating_degree_days', 0.014425544350508142),
 ('air_temperature', 0.013671210212765756),
 ('health_insurance_none_rate', 0.013651241463206006),
 ('average_household_size', 0.013279138760968026),
 ('earth_temperature', 0.012607313920744097)]

## Get Predictions for Whole Dataset

In [41]:
predictions_final = model_final.transform(regressor_df)

In [42]:
predictions_final.show(50)

+--------------------+--------------------+--------------------+--------------------+
|               label|            features|     indexedFeatures|          prediction|
+--------------------+--------------------+--------------------+--------------------+
|                 0.0|[70352.7898693999...|[70352.7898693999...|0.002132466122110...|
|     0.0067264573991|[61727.0852018,0....|[61727.0852018,0....|0.002178753071469...|
|0.001111934766490...|[71496.8865827,0....|[71496.8865827,0....|0.002068256883884308|
|                 0.0|[86840.1527549999...|[86840.1527549999...|0.003109883510450327|
|0.002086593635890...|[89135.3155973000...|[89135.3155973000...|0.002646366771488...|
|                 0.0|[62225.9036145,0....|[62225.9036145,0....|0.002052899294023...|
|    0.00425531914894|[41068.9361701999...|[41068.9361701999...|0.003449034322424463|
|                 0.0|[74073.8336714,0....|[74073.8336714,0....|0.002052899294023...|
|                 0.0|[69412.1924348,0....|[69412.1924

In [45]:
final_preds=predictions_final.select('prediction').collect()
final_preds_list=[i.prediction for i in final_preds]
final_preds_list[0:10]

[0.0021324661221101176,
 0.0021787530714694275,
 0.002068256883884308,
 0.003109883510450327,
 0.0026463667714880456,
 0.0020528992940237077,
 0.003449034322424463,
 0.0020528992940237077,
 0.002068256883884308,
 0.0022854204215557923]

In [53]:
#Set infinity and blank spaces to NaN in independent variables, set infinite to N/A in dependent variable
solar_full=solar[features].replace([np.inf,' '],np.nan)
solar_full['number_of_solar_system_per_household']=solar['number_of_solar_system_per_household'].replace(np.inf,np.nan)
solar_full['fips']=solar['fips']
solar_full=solar_full.loc[np.isfinite(solar_full['number_of_solar_system_per_household'])]
solar_full=solar_full.assign(predicted_solar=final_preds_list)
solar_full.head(10)

,average_household_income,gini_index,heating_fuel_coal_coke,heating_fuel_electricity,heating_fuel_fuel_oil_kerosene,heating_fuel_gas,heating_fuel_housing_unit_count,heating_fuel_none,heating_fuel_other,heating_fuel_solar,...,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,number_of_solar_system_per_household,fips,predicted_solar
0,70352.789869,0.3490,200,448,98,1720,2527,10,51,0,...,34,0,0,25,12,0,9.46,0.000000,27145011200,0.002132
1,61727.085202,0.4074,20,379,17,1799,2230,0,15,0,...,34,0,0,25,12,0,9.46,0.006726,27145011301,0.002179
2,71496.886583,0.3926,69,440,45,2098,2698,29,17,0,...,34,0,0,25,12,0,9.46,0.001112,27145011302,0.002068
3,86840.152755,0.3949,188,442,61,1113,1833,13,16,0,...,34,0,0,25,12,0,9.46,0.000000,27145011304,0.003110
4,89135.315597,0.4463,96,497,47,1202,1917,31,44,0,...,34,0,0,25,12,0,9.46,0.002087,27145011400,0.002646
5,62225.903614,0.3847,72,759,81,1695,2656,45,4,0,...,34,0,0,25,12,0,9.46,0.000000,27145011500,0.002053
6,41068.936170,0.7166,3,222,5,193,470,16,31,0,...,34,0,0,25,12,0,9.46,0.004255,27145011600,0.003449
7,74073.833671,0.4317,108,382,190,783,1479,3,13,0,...,34,0,0,25,12,0,9.46,0.000000,27145010500,0.002053
8,69412.192435,0.3824,241,534,156,1735,2723,20,37,0,...,34,0,0,25,12,0,9.46,0.000000,27145011100,0.002068
9,82502.407069,0.3588,18,734,27,2419,3282,29,55,0,...,34,0,0,25,12,0,9.46,0.002742,27145010102,0.002285


In [56]:
relevant_fields=['number_of_solar_system_per_household']+features+['fips']
final_frame=pd.merge(solar[relevant_fields],solar_full[['fips','predicted_solar']],on=['fips'],how='left')
final_frame.head(10)

,number_of_solar_system_per_household,average_household_income,gini_index,heating_fuel_coal_coke,heating_fuel_electricity,heating_fuel_fuel_oil_kerosene,heating_fuel_gas,heating_fuel_housing_unit_count,heating_fuel_none,heating_fuel_other,...,diversity,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,fips,predicted_solar
0,0.000000,70352.789869,0.3490,200,448,98,1720,2527,10,51,...,0.046960,34,0,0,25,12,0,9.46,27145011200,0.002132
1,0.006726,61727.085202,0.4074,20,379,17,1799,2230,0,15,...,0.145934,34,0,0,25,12,0,9.46,27145011301,0.002179
2,0.001112,71496.886583,0.3926,69,440,45,2098,2698,29,17,...,0.009150,34,0,0,25,12,0,9.46,27145011302,0.002068
3,0.000000,86840.152755,0.3949,188,442,61,1113,1833,13,16,...,0.187334,34,0,0,25,12,0,9.46,27145011304,0.003110
4,0.002087,89135.315597,0.4463,96,497,47,1202,1917,31,44,...,0.090766,34,0,0,25,12,0,9.46,27145011400,0.002646
5,0.000000,62225.903614,0.3847,72,759,81,1695,2656,45,4,...,0.081981,34,0,0,25,12,0,9.46,27145011500,0.002053
6,0.004255,41068.936170,0.7166,3,222,5,193,470,16,31,...,0.470443,34,0,0,25,12,0,9.46,27145011600,0.003449
7,0.000000,74073.833671,0.4317,108,382,190,783,1479,3,13,...,0.008239,34,0,0,25,12,0,9.46,27145010500,0.002053
8,0.000000,69412.192435,0.3824,241,534,156,1735,2723,20,37,...,0.029574,34,0,0,25,12,0,9.46,27145011100,0.002068
9,0.002742,82502.407069,0.3588,18,734,27,2419,3282,29,55,...,0.120052,34,0,0,25,12,0,9.46,27145010102,0.002285


In [58]:
final_frame.to_csv('baseline_model_predictions.csv')